# SimPy

## Install Dependencies

In [5]:
!pip install -U simpy

## Import Dependenies

In [6]:
import simpy
from simpy import Environment
from typing import Generator, Optional, Union
from simpy.events import Event
import random
import statistics
import numpy as np

## Call Center Simulation

In [7]:
class Simulation(Environment):
    def __init__(self,
                 name: str,
                 until: Optional[Union[Union[int, float], Event]] = None) -> None:
        super().__init__()
        print(f'Starting simulation "{name}" at {self.now:.2f}.')
        self.process(self.process_generator())
        self.run(until=until)
        print(f'Ending simulation "{name}" at {self.now:.2f}.')

    def process_generator(self) -> Generator:
        raise NotImplementedError(f'The method "process_generator" should be overridden in the derived class.')

In [8]:
class CallCenter(Simulation):
    customers_handled: int = 0

    def __init__(self,
                 max_runtime_mins: int,
                 n_call_center_agents: int,
                 avg_support_time_in_mins: int,
                 customer_interval_in_min: int) -> None:
        # Assign resources.
        self.avg_support_time_in_mins = avg_support_time_in_mins
        self.customer_interval_in_min = customer_interval_in_min
        self.call_center_agent_resources = simpy.Resource(env=self, capacity=n_call_center_agents)

        # Initiate the simulation.
        super().__init__(name='Call Center', until=max_runtime_mins)

    def __support_customer__(self, customer_name: str) -> Generator:
        random_time = max(1, np.random.normal(self.avg_support_time_in_mins, 4))

        yield self.timeout(delay=random_time)
        print(f'Support finished for {customer_name} at {self.now:.2f}.')

    def __take_call__(self, customer_name: str) -> Generator:
        print(f'Customer "{customer_name}" enters waiting queue at {self.now:.2f}.')
    
        with self.call_center_agent_resources.request() as request:
            # Wait for a call center agent to become available.
            yield request
            print(f'Customer "{customer_name}" enters call at {self.now:.2f}.')

            # Handle the support item associated with the call.
            yield self.process(self.__support_customer__(customer_name=customer_name))
            print(f'Customer "{customer_name}" left call at {self.now:.2f}.')

            self.customers_handled += 1
    
    def process_generator(self) -> Generator:
        for i in range(1, 6):
            self.process(self.__take_call__(customer_name=i))

        while True:
            yield self.timeout(delay=random.randint(self.customer_interval_in_min-1, self.customer_interval_in_min+1))
            i += 1
            self.process(self.__take_call__(customer_name=i))

In [9]:
sim = CallCenter(max_runtime_mins=120,
                 n_call_center_agents=2,
                 avg_support_time_in_mins=5,
                 customer_interval_in_min=5)

print(f'The call center handled {sim.customers_handled} customer calls.')

Starting simulation "Call Center" at 0.00.
Customer "1" enters waiting queue at 0.00.
Customer "2" enters waiting queue at 0.00.
Customer "3" enters waiting queue at 0.00.
Customer "4" enters waiting queue at 0.00.
Customer "5" enters waiting queue at 0.00.
Customer "1" enters call at 0.00.
Customer "2" enters call at 0.00.
Support finished for 2 at 1.00.
Customer "2" left call at 1.00.
Customer "3" enters call at 1.00.
Support finished for 1 at 1.07.
Customer "1" left call at 1.07.
Customer "4" enters call at 1.07.
Support finished for 4 at 2.07.
Customer "4" left call at 2.07.
Customer "5" enters call at 2.07.
Support finished for 5 at 3.07.
Customer "5" left call at 3.07.
Customer "6" enters waiting queue at 5.00.
Customer "6" enters call at 5.00.
Support finished for 3 at 5.45.
Customer "3" left call at 5.45.
Customer "7" enters waiting queue at 11.00.
Customer "7" enters call at 11.00.
Support finished for 6 at 13.03.
Customer "6" left call at 13.03.
Customer "8" enters waiting qu